In [7]:
from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import OpenAI, SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
import psycopg2
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.callbacks import get_openai_callback
from langchain.chains import create_sql_query_chain
from sqlalchemy import create_engine, MetaData
from llama_index import LLMPredictor, ServiceContext, SQLDatabase, VectorStoreIndex
from llama_index.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.objects import SQLTableNodeMapping, ObjectIndex, SQLTableSchema
from langchain import OpenAI
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain


from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd
import os

from dotenv import load_dotenv

load_dotenv()

True

In [8]:
# get env variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [9]:
model = GPT4All(model='ggml-model-gpt4all-falcon-q4_0.bin')

Found model file at  /Users/Samuel/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin


objc[25105]: Class GGMLMetalClass is implemented in both /Users/Samuel/Library/Caches/pypoetry/virtualenvs/chatbot-clutch-KOPEQSmf-py3.11/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x17e5a8228) and /Users/Samuel/Library/Caches/pypoetry/virtualenvs/chatbot-clutch-KOPEQSmf-py3.11/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x17e6c0228). One of the two will be used. Which one is undefined.


falcon_model_load: loading model from '/Users/Samuel/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


In [10]:
context = """
You are a friendly loan assistant from a lending startup that responds in a conversational
manner to users questions. Keep the answers short, unless specifically
asked by the user to elaborate on something. Always maintain a polite and positive sentiment tone.

You are only allowed to answer questions about the loan, for any other scope avoid to answer the question. 

The user wants to know whether the loan is approved, decline, or awaiting a
decision.

The user is also asking about the associated terms of the loan (APR, rate, term,
and amount approved, as well as monthly payment).

For any other question from the user, the model should still be polite and courteous but
suggest that a more senior lending officer may need to answer that question. And when
in doubt, you should avoid answering the question.

"""

In [11]:
template = f"""
Please use the following context to answer questions.
Context: {context}
"""

In [12]:
model(template)

'{\n"user_question": {\n"question": {\n"type": {\n"value": "loan",\n"description": "What type of loan are you applying for?"\n}\n},\n"details": {\n"type": {\n"value": "personal",\n"description": "What type of loan are you applying for?"\n}\n}\n}'

In [13]:
db = SQLDatabase.from_uri("sqlite:///../data/clutch_database.db")

In [129]:
template = """
Write a SQL Query given the table named {Table} and columns as a list {Columns} for the given question : 
{question}.

All variables are independent, no need to call all of them in the query.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])

llm_chain = LLMChain(prompt=prompt, llm=model)

def get_llm_response(table: str, question: str,cols: (list|str)) -> str:
    llm_chain = LLMChain(prompt=prompt, 
                         llm=model
                         )
    response= llm_chain.run({"Table" : table,"question" :question, "Columns" : cols})
    return response

In [135]:
table = "clients"
cols = ['Status', 'ID',  'ModificationDate', 'Rate', 'Amount']
question = "What is the status from client 1?"
get_llm_response(table,question,cols)

'SELECT * FROM clients WHERE ID = 1;'

In [136]:
db.run(get_llm_response(table,question,cols))

"[(1, 'Initial', 'Counter', None, '1', 'System Decision', None, 1, 3.3154475537542725, 22465.3, 36.92491288070312, 'C', '948', '\\n', '\\n', '\\n', '\\n', '\\n', '\\n', 1678287124)]"

In [21]:
type(prompt)

langchain.prompts.prompt.PromptTemplate

In [44]:
db.run("select * from clients")

"[(2, 'Final', 'Declined', None, '678901.0', 'System Decision', 1692136800, 1, 0.0, 0.0, 0.0, None, '789.0', '\\n      ', '\\n      ', '\\n      ', '\\n      ', '\\n      ', '\\n      '), (2, 'Reviewed', 'Approved', None, '87654321.0', 'System Decision', 1692136800, 1, 2.875, 45000.0, 72.0, 'B', '123.0', '\\n      ', '\\n      ', '\\n      ', '\\n      ', '\\n      ', '\\n      '), (2, 'Initial', 'Counter', None, '678901.0', 'System Decision', 1692136800, 1, 4.125, 20000.0, 48.0, 'C', '456.0', '\\n      ', '\\n      ', '\\n      ', '\\n      ', '\\n      ', '\\n      '), (2, 'Initial', 'Counter', None, '654321.0', 'System Decision', 1692136800, 1, 3.75, 18000.0, 60.0, 'B', '567.0', '\\n      ', '\\n      ', '\\n    ', '\\n      ', '\\n      ', '\\n      '), (0, 'Reviewed', 'Approved', None, '0', 'System Decision', 1655330400, 1, 5.284204437909292, 23393.44855969129, 36.946564605424314, 'C', '941', '\\n', '\\n', '\\n', '\\n', '\\n', '\\n'), (1, 'Reviewed', 'Approved', None, '1', 'System

In [ ]:
db_chain = SQLDatabaseChain.from_llm(model, db, verbose=True, use_query_checker=True)

In [ ]:
db_chain.run("how many rows are in the table clients?")

In [ ]:
type("how many rows are in the table clients?")

In [16]:
toolkit = SQLDatabaseToolkit(db=db, llm=model)

template = """
Write a SQL Query given the table named {Table} and columns as a list {Columns} for the given question : 
{question}.

All variables are independent, no need to call all of them in the query.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])

agent_executor = create_sql_agent(
    llm=model,
    toolkit=toolkit,
    verbose=True,
    prompt=prompt
)

In [17]:
query = "how many rows are in the table clients?"
with get_openai_callback() as cb:
    res = agent_executor.run(query)
    print("SQL Query Result:")
    print(res)
    print("")
    print(cb)



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Action: sql_db_query_checke`

In [61]:
chain = create_sql_query_chain(model, db)
response = chain.invoke({"question":"create a query to count rows in a table named clients"})
print(response)

TypeError: fromisoformat: argument must be str

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [ ]:
agent_executor = create_sql_agent(
    llm=model,
    toolkit=toolkit,
    verbose=True
)

In [30]:
PROMPT = """ 
Given an input question, first create a syntactically correct sql query to run,  
then look at the results of the query and return the answer.  
The question: {question}
"""

In [ ]:
question = "What is the status of the client of ID=1 ?"

In [ ]:
agent_executor.run(PROMPT.format(question=question))

In [ ]:
print(model("create a sql query to return the number of lines of the table named clients, only the query."))

In [ ]:
questions = pd.read_csv("../data/processed/questions.csv")

In [ ]:
questions = questions.iloc[:15]

In [ ]:
dict_bool = {True: '', False: 'not'}

for idx, row in questions.iterrows():
    print(f"{idx+1}/{len(questions)}")
    model(f"To give you more context, you can{dict_bool[row[1]]} answer the following question about the loan: {row[0]}")


In [ ]:
model("What documents are required to apply for a loan?	")

In [ ]:
questions

In [47]:
model("""Instruction: you cannot answer the question: What documents are required to apply for a loan?

Question: What documents are required to apply for a loan?	

""")

'Answer: To apply for a loan, you will typically need to provide proof of income (such as pay stubs or tax returns), proof of employment (such as W-2s or pay stubs), and proof of assets (such as bank statements or investment account statements). You may also be required to provide additional documents depending on the type of loan you are applying for.'

In [ ]:
model()

In [19]:
" Write a SQL Query given the table named {Table} and columns as a list {Columns} for the given question : {question}. All variables are independent, no need to call all of them in the query."
   

' Write a SQL Query given the table named {Table} and columns as a list {Columns} for the given question : {question}. All variables are independent, no need to call all of them in the query.'